# Test duration

In [102]:
import numpy as np
import scipy.stats as sps
import pandas as pd
import plotly.graph_objs as go

In [92]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    z = sps.norm.ppf(1 - alpha/2) + sps.norm.ppf(1-beta)
    n = (k+1) * z ** 2 * (sigma_1 ** 2 + sigma_2 **2 / k) / (delta_effect ** 2)
    return n

In [96]:
def power_of_a_test(dist, alpha, params_1, params_2):
    p_values_ttest = []
    p_values_mannwhitneyu = []
    n_exp = 10000
    for i in range(n_exp):
        x_a = dist(**params_1)
        x_b = dist(**params_2)
        p_values_ttest.append(sps.ttest_ind(x_a, x_b, equal_var=False).pvalue)
        p_values_mannwhitneyu.append(sps.mannwhitneyu(x_a, x_b).pvalue)
    p_values_ttest = np.array(p_values_ttest)
    p_values_mannwhitneyu = np.array(p_values_mannwhitneyu)
    print(f'ttest power: {p_values_ttest[p_values_ttest < alpha].shape[0] / p_values_ttest.shape[0] * 100}')
    print(f'mannwhitneyu power: {p_values_mannwhitneyu[p_values_mannwhitneyu < alpha].shape[0] / p_values_mannwhitneyu.shape[0] * 100}')

In [103]:
def get_qq_plot(p_values, title):
    p_values = np.array(p_values)
    probs = []
    x = [0.01 * i for i in range(101)]
    for i in range(101):
        alpha_step = 0.01 * i
        probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])
    fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
    fig.update_layout(height=600, width=600, title=title) 
    return fig

In [97]:
powers = [0.8, 0.6]
alpha = 0.05
p = 0.2
effects = [0.03, 0.3]
for power, effect in zip(powers, effects):
    sigma_1 = np.sqrt(p * (1 - p))
    sigma_2 = np.sqrt((p + effect)  * (1 - p - effect))
    test_duration = duration(k=1, delta_effect=effect, sigma_1=sigma_1, sigma_2=sigma_2, alpha=alpha, beta=1 - power)
    print(f'effect: {effect}, power: {power}, duration: {test_duration}')
    power_of_a_test(stats.bernoulli.rvs, alpha, dict(p=p, size=int(test_duration/2)), dict(p=p + effect, size=int(test_duration/2)))
    print(f'expected_power: {power}')

effect: 0.03, power: 0.8, duration: 5879.68301877573
ttest power: 79.83
mannwhitneyu power: 79.82000000000001
expected_power: 0.8
effect: 0.3, power: 0.6, duration: 44.63301884532971
ttest power: 55.48
mannwhitneyu power: 55.48
expected_power: 0.6


![Test 1](test_1.png)
![Test 2](test_2.png)

#  Linearization

In [152]:
def linearization(p, effect):
    n_exp = 1000
    p_values = []
    p_values_lin = []
    for _ in range(n_exp):
        records = []
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=100, scale=100))
            clicks = sps.bernoulli.rvs(p=p, size=n_views)
            records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "A"])
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=100, scale=100))
            clicks = sps.bernoulli.rvs(p=p + effect, size=n_views)
            records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "B"])
        df_data = pd.DataFrame(records, columns=["views", "clicks", "cr", "group"])

        cr_A = df_data[df_data["group"] == "A"]["clicks"].sum() / df_data[df_data["group"] == "A"]["views"].sum()
        df_data["cr_lin"] = df_data["clicks"] - cr_A * df_data["views"]

        x_a = df_data[df_data["group"] == "A"]["cr"]
        x_b = df_data[df_data["group"] == "B"]["cr"]
        p_value = sps.ttest_ind(x_a, x_b).pvalue
        p_values.append(p_value)

        x_a_lin = df_data[df_data["group"] == "A"]["cr_lin"]
        x_b_lin = df_data[df_data["group"] == "B"]["cr_lin"]
        p_value_lin = sps.ttest_ind(x_a_lin, x_b_lin).pvalue
        p_values_lin.append(p_value_lin)
    return p_values, p_values_lin

In [153]:
p_values, p_values_lin = linearization(0.05, 0)

In [154]:
get_qq_plot(p_values, title='without Linearization')

In [155]:
get_qq_plot(p_values_lin, title='with Linearization')

In [156]:
p_values, p_values_lin = linearization(0.05, 0.005)

In [157]:
p_values = np.array(p_values)
print(f'power without Linearization: {p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100}')

power without Linearization: 52.800000000000004


In [158]:
p_values_lin = np.array(p_values_lin)
print(f'power with Linearization: {p_values_lin[p_values_lin < alpha].shape[0] / p_values_lin.shape[0] * 100}')

power with Linearization: 56.99999999999999


# CUPED

In [211]:
def cuped(dist, pre_exp, params_1, params_2):
    n_exp = 1000
    p_values = []
    p_values_cuped = []

    # Pre_experiment
    for _ in range(n_exp):
        df_A = pd.DataFrame()
        df_A["user"] = [f"A_{x:5}" for x in range(size)]
        df_A["pre_exp"] = pre_exp
        df_B = pd.DataFrame()
        df_B["pre_exp"] = pre_exp
        df_B["user"] = [f"B_{x:5}" for x in range(size)]
        df_A["payments"] = dist(**params_1) * df_A["pre_exp"]
        df_B["payments"] = dist(**params_2) * df_B["pre_exp"]

        p_values.append(sps.ttest_ind(df_A["payments"], df_B["payments"]).pvalue)

        x_a = df_A["pre_exp"]
        x_b = df_B["pre_exp"]
        y_a = df_A["payments"]
        y_b = df_B["payments"]
        theta = np.cov(x_a, y_a)[0,1] / np.std(x_a)**2

        df_A["payments_cuped"] = df_A["payments"] - theta * df_A["pre_exp"]
        df_B["payments_cuped"] = df_B["payments"] - theta * df_B["pre_exp"]

        p_values_cuped.append(sps.ttest_ind(df_A["payments_cuped"], df_B["payments_cuped"]).pvalue)
    return p_values, p_values_cuped

In [212]:
size = 10000
pre_exp = sps.norm.rvs(loc=100, scale=20, size=size)
p_values, p_values_cuped = cuped(sps.expon.rvs, pre_exp,
                                 dict(loc=100, scale=100, size=size), dict(loc=100, scale=100, size=size))

In [213]:
get_qq_plot(p_values_cuped, title='Cuped')

In [214]:
effect = 0.005
p_values, p_values_cuped = cuped(sps.expon.rvs, pre_exp,
                                 dict(loc=1, scale=0.1, size=size), dict(loc=1 + effect, scale=0.1, size=size))

In [215]:
p_values = np.array(p_values)
print(f'power ordinal test: {p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100}')

power ordinal test: 13.200000000000001


In [216]:
p_values_cuped = np.array(p_values_cuped)
print(f'power with CUPED: {p_values_cuped[p_values_cuped < alpha].shape[0] / p_values_cuped.shape[0] * 100}')

power with CUPED: 94.69999999999999
